## Facebook Graph API<span style="color:blue"> | Explorer</span>

[Facebook Graph API Explorer](https://developers.facebook.com/tools/explorer)

### components of the API
+ according to official doc:
    + **nodes** - basically "things" such as a User, a Photo, a Page, a Comment.
    + **edges** - the connections between those "things", such as a Page's Photos, or a Photo's Comments
    + **fields** - info about those "things", such as the birthday of a User, or the name of a Page

### basic syntax
`<node_id>/<edge_name>?fields=f1,f2,...`
+ a node can be a person, a photo, a page, ...
    + different nodes have their corresponding fields
    + any field is also comprised by nodes, e.g., the field photo is comprised by photo nodes
+ the `me` node is a special node pointing to the owner of the current token
+ example:
    + `me/photos?fields=from`

### which field?
+ use `metadata=1` to extract all possible fields on a node
    + `<node_id>?metadata=1`
+ alternatively, use the interactive "Search for a field" GUI tool to the left panel on the api explorer
+ not all fields will be returned in an API call

### nested query
`<node_id>?fields=f1{subf1,subf2,...},f2,...`

or

`<node_id>?fields=f1.fields(subf1,subf2,...),f2,...`
+ there is no limit on the depth of nesting (according to official doc)

### node or edge?
+ edge is the population, node is talking about individual
+ *all* of my photos => edge (no id); *this* photo of mine => node (with id)
+ fields are defined on nodes, not edges
+ any difference?
    + `me?fields=photos{from}`
    + `me/photos?fields=from`

### modifier: limit
+ optionally, use method `.limit(n)` or `?limit=n` to restrict the number of objects returned
    + on a field:
        + `me?fields=photos.limit(10)`
    + on an edge:
        + `me/photos?limit=10`

### modifier: summary
+ fields (or edges) with multiple entities usually have a summary containing their counts
    + on a field:
        + `<photo_id>?fields=likes.summary(true)`
    + on an edge:
        + `<photo_id>/likes?summary=true`

### modifier chaining
+ on a field:
    + `<photo_id>?files=likes.limit(1).summary(true)`
+ on an edge:
    + `<photo_id>/likes?limit=1&summary=true`

### paging
obviously, not all results are returned in one shut (by design)

### scenario: who likes my posts?
+ start from the `me` node
+ what fields a post can have?
    + use `?metadata=1` on any one of your post (try `me/posts` first)
    + "likes" are indeed edges (connections) instead of fields
    + still, we can use it in the field query (just as `me/photos` to `me?fields=photos`)
+ setup a time range by using the `since` modifier => [timestamp converter](http://www.epochconverter.com)
+ get total count using the `summary` modifier
+ finally, something like this:
    + `me/posts?fields=likes.fields(name).summary(true)&since=1420070400&limit=100`

## Facebook Graph API<span style="color:blue"> | Programmatic</span>

### go beyond the API explorer...
+ an API query is nothing more than a HTTP request
    + with valid access_token, of course
    + and the returned data are always json
+ use "Get Code" button in the explorer and look for the "cURL" tab

```
curl -i -X GET \
 "https://graph.facebook.com/v2.4/me?fields=context&access_token=CAACEdEose0cBAL34H6NiMZB3ZCnPaZBnShQoSY9GZCh81kDLbQZArxKGEPY981H7KfBUjG99jThga2OxQ7owu03IZCgoEcjMDmVSyeZAzos3JZBvWEzbRbfX0DZAl0Au2ybbbZCNZBOsZCYGmjKqCLyTHftwrnOerU07Pismb3QBxYommKEo7oGsWTIIREpbKu4VlHMJY7Q7ZBY00aAZDZD"
```

### get all metadata of a node

In [175]:
import requests
api_addr = "https://graph.facebook.com/v2.4/me"
token = "CAACEdEose0cBAPGFtYmvw8Ij1PotlLE9X5OnFAuyQYHYiYZBr93fj9qcUkOwgDB2sWiMZCSUckIrfZCM1ddQoRT6mZAdH6RQCU7rWjj1JlVjlZCdPS5KevvcdjAZCJZBHQTcQTTSPaWqOjcvZAcYtWBK6YmbF8xFTtOoOTCqW7nupUy0SDhwJ5ILxW1HTWVD9TqeW4Szfh3AOgZDZD"
qs = {"metadata": 1, 
      "access_token": token}
r = requests.get(api_addr, params=qs)
meta = r.json()

In [176]:
print meta.keys()

[u'id', u'name', u'metadata']


In [177]:
print meta["metadata"].keys()

[u'connections', u'fields', u'type']


In [178]:
metafields = [ l["name"] for l in meta["metadata"]["fields"] ]
for s in metafields:
    print s

id
about
age_range
bio
birthday
context
currency
devices
education
email
favorite_athletes
favorite_teams
first_name
gender
hometown
inspirational_people
install_type
installed
interested_in
is_shared_login
is_verified
languages
last_name
link
location
locale
meeting_for
middle_name
name
name_format
payment_pricepoints
test_group
political
relationship_status
religion
security_settings
significant_other
sports
quotes
third_party_id
timezone
token_for_business
updated_time
shared_login_upgrade_required_by
verified
video_upload_limits
viewer_can_send_gift
website
work
public_key
cover


### traverse paginated json

In [179]:
import requests
def getAllLikes(token, node):
    result = []
    api_addr = "https://graph.facebook.com/v2.4/%s/likes" % node
    qs = {"fields": "name", 
          "access_token": token}
    r = requests.get(api_addr, params=qs)
    res = r.json()
    if not len(res["data"]):
        return result
    else:
        result += res["data"]
        while "next" in res["paging"]:
            cursor_next = res["paging"]["cursors"]["after"]
            qs["after"] = cursor_next
            r = requests.get(api_addr, params=qs)
            res = r.json()
            result += res["data"]
        return result
    
result = getAllLikes(token="CAACEdEose0cBAPGFtYmvw8Ij1PotlLE9X5OnFAuyQYHYiYZBr93fj9qcUkOwgDB2sWiMZCSUckIrfZCM1ddQoRT6mZAdH6RQCU7rWjj1JlVjlZCdPS5KevvcdjAZCJZBHQTcQTTSPaWqOjcvZAcYtWBK6YmbF8xFTtOoOTCqW7nupUy0SDhwJ5ILxW1HTWVD9TqeW4Szfh3AOgZDZD", 
                     node="100000862115668_932029203502475")

In [180]:
result

[{u'id': u'315371435293768', u'name': u'Tindy Cheng'},
 {u'id': u'969673423047100', u'name': u'ChunKuei Chu'},
 {u'id': u'756609997705112', u'name': u'\u6797\u744b\u744b'},
 {u'id': u'852448504770317', u'name': u'Mark Yang'},
 {u'id': u'1487000218', u'name': u'\u8607\u4e2d\u624d'},
 {u'id': u'1161704643846945', u'name': u'\u9673\u667a\u6cd3'},
 {u'id': u'10204030571792411', u'name': u'Mansun Kuo'},
 {u'id': u'10152783776360960', u'name': u'Jim Pai'},
 {u'id': u'748835111857876', u'name': u'Yi LinWei'},
 {u'id': u'788499557828349', u'name': u'Wush Wu'},
 {u'id': u'10200724712359727', u'name': u'Cedar Su'},
 {u'id': u'10202285354957426', u'name': u'\u67ef\u9d3b\u5100'},
 {u'id': u'878903608788910', u'name': u'\u9b4f\u5ef7\u65ed'},
 {u'id': u'1182843781729426', u'name': u'\u6d2a \u5fd7\u6587'},
 {u'id': u'912921898725737', u'name': u'\u77f3\u5c0f\u77f3'},
 {u'id': u'10153331968758409', u'name': u'Yencheng Chen'},
 {u'id': u'10152159007283224', u'name': u'\u6797\u76df\u5091'},
 {u'id': u'8

In [181]:
# check for zero-liked post
check = getAllLikes(token="CAACEdEose0cBAPGFtYmvw8Ij1PotlLE9X5OnFAuyQYHYiYZBr93fj9qcUkOwgDB2sWiMZCSUckIrfZCM1ddQoRT6mZAdH6RQCU7rWjj1JlVjlZCdPS5KevvcdjAZCJZBHQTcQTTSPaWqOjcvZAcYtWBK6YmbF8xFTtOoOTCqW7nupUy0SDhwJ5ILxW1HTWVD9TqeW4Szfh3AOgZDZD", 
                    node="100000862115668_930036487035080")
check

[]

### scenario:

TBC

### programmatically get the token?

## Regular Expression

### what is re?
from wikipedia:
> a regular expression (abbreviated regex or regexp and sometimes called a rational expression) is a sequence of characters that define a search pattern, mainly for use in pattern matching with strings, or string matching, i.e. "find and replace"-like operations.

### why re?
imagine there is a dataset looks like this...
> kyle chung; (02)2256-1116; New Taipei City 
<br>
Mr. Funghi,Earth Anywhere, 07-21180293
<br>
free tel: 0800 000 123 #1234
<br>
2945 1412  street no.123  Dr.A(not Dr.B)

YOUR BOSS: "Give me all the telphone numbers only." 

In [182]:
import re

test_str = ["kyle chung; (02)2256-1116; New Taipei City",
            "Mr. Funghi,Earth Anywhere, 07-21180293",
            "free tel: 0800 000 123 #1234",
            "Dr.A (not Dr.B) street no.123 2945 1412"]

def giveMeTel(pattern, test_str=test_str):
    p = re.compile(pattern)
    for s in test_str:
        m = p.search(s)
        if m:
            print m.group()


In [183]:
# 1st try: get all digits
    
giveMeTel("[0-9]")

0
0
0
1


In [184]:
# 2nd try: get all digits, not just one
    
giveMeTel("[0-9]+")

02
07
0800
123


In [185]:
# 3rd try: get all digits, not just one, and dash
    
giveMeTel("[0-9-]+")

02
07-21180293
0800
123


In [186]:
# 4th try: get all digits, not just one, and dash and paranthesis and blanks...
    
giveMeTel("[0-9-()]+")

(02)2256-1116
07-21180293
0800
(


In [187]:
# now allow blanks, oops!
    
giveMeTel("[0-9-() ]+")

 
 
 
 (


In [188]:
# !(@#$&!@#*$!(@&%(!@*...
    
giveMeTel("\(?[0-9]+\)?")

(02)
07
0800
123


In [189]:
giveMeTel("\(?[0-9]+\)?[0-9-]+")

(02)2256-1116
07-21180293
0800
123


In [190]:
giveMeTel("\(?[0-9]+\)?[0-9-]+")

(02)2256-1116
07-21180293
0800
123


In [191]:
giveMeTel("\(?[0-9]+\)?[0-9- ]+")

(02)2256-1116
07-21180293
0800 000 123 
123 2945 1412


In [192]:
giveMeTel("\(?[0-9]{0,2}\)?-?[0-9]{4}[ -]?[0-9]{4}")

(02)2256-1116
07-21180293
2945 1412


In [193]:
giveMeTel("[0-9]{4}[ ]?[0-9]{3}[ ]?[0-9]{3}")

0800 000 123


In [194]:
giveMeTel("\(?[0-9]{0,2}\)?-?[0-9]{4}[ -]?[0-9]{4}|[0-9]{4}[ ]?[0-9]{3}[ ]?[0-9]{3}")

(02)2256-1116
07-21180293
0800 000 123
2945 1412


In [195]:
giveMeTel("\(?[0-9]{0,2}\)?-?[0-9]{4}[ -]?[0-9]{4}|[0-9]{4}[ ]?[0-9]{3}[ ]?[0-9]{3}[ ]?#?[0-9]+")

(02)2256-1116
07-21180293
0800 000 123 #1234
2945 1412


### components of re
+ characters (string-literals)
    + ABCabc123
+ meta-characters (operators)
    + `. ^ $ * + ? { } [ ] \ | ( )`
+ special sequences (short-cuts)
    + `\d \D \s \S \w \W \b \B`

#### wildcarder: `.`
match anything except newline(`\n`)

#### bounders: `^ $`
+ `^`: (following) the beginning character
    + `^this` matches anything started with "this"
+ `$`: (preceding) the ending character
    + `that$` matches anything ended with "that"

#### repeaters: `* + ?`
to specify the occurrence condition of its previous character
+ `*`: any occurrence (including zero)
+ `+`: at least one occurrence
+ `?`: zero or exactly one occurrence

#### advanced repeaters: `{m,n}`
at least `m` and at most `n` occurrence

example:
+ `a{1,3}`: character "a" must occur at least one and at most three time
+ `a{1,}`: character "a" must occur at least one time
+ `a{,3}`: character "a" must occur at most three time

#### `[]`: character class
+ to specify a set of characters that may occcur
+ most meta-char will lose their speacial meaning within the class
    + `[0-9$]` matches any of numbers 0 to 9 or the dollar sign($)
    + here the bounder meta-char `$` is treated as-is
+ special meaning only triggered within the class:
    + `-`: ranger, e.g., `[0-9a-zA-Z]`
    + `^`: negator, e.g., `[^0-9]`

#### or: `|`
example:
+ `abc|def`: either "abc" or "def"

#### grouper: `()`
example:
+ `(abc)+`: at least one occurrence of "abc"
+ `abc+`: here the repeater `+` only operates on character "c"

#### special sequences
short-cut of specific character classes
+ `\d`: any decimal digit => `[0-9]`
+ `\D`: any non-digit => `[^0-9]`
+ `\s`: any white space => `[ \t\n\r\f\v]`
+ `\S`: any non-white space => `[^ \t\n\r\f\v]`
+ `\w`: any alphanumeric => `[a-zA-Z0-9_]`
+ `\W`: any non-alphanumeric => `[^a-zA-Z0-9_]`
+ `\b`: 

#### escaper: `\`
to have its following character **as-is**, i.e., stripping special meaning

this is required when you want to match literal meta-char

## Regex in Python<span style="color:blue"> | Basic</span>

### import python re module

In [196]:
import re

### compile a regex

In [197]:
p = re.compile("test+")
# return a pattern object
print p
# check methods
[me for me in dir(p) if callable(getattr(p, me)) and not "__" in me]

['findall', 'finditer', 'match', 'scanner', 'search', 'split', 'sub', 'subn']

#### the `match` method
to perform pattern matching **from the beginning** of a string

In [198]:
print p.match("123")

None


In [199]:
print p.match("123test")

None


In [200]:
print p.match("tes")

None


In [201]:
print p.match("test")

In [202]:
print p.match("testtttt")

#### the `match` object

In [203]:
m = p.match("testtttt")
[me for me in dir(m) if callable(getattr(m, me)) and not "__" in me]

['end', 'expand', 'group', 'groupdict', 'groups', 'span', 'start']

In [204]:
# return the matched string
m.group()

'testtttt'

In [205]:
"the match starts at pos %s and ends at pos %s" % (m.start(), m.end())

'the match starts at pos 0 and ends at pos 8'

In [206]:
"the match starts at pos %s and ends at pos %s" % m.span()

'the match starts at pos 0 and ends at pos 8'

#### the `search` method
to perform pattern matching **from anywhere** of a string

In [207]:
m = p.search("123test456")
if m:
    print "pattern matched at pos %s to pos %s" % m.span()
else:
    print "no match"

pattern matched at pos 3 to pos 7


#### the `findall` and `finditer` method

In [208]:
p = re.compile("\d")
p.findall("24 hours a day; 8 in sleep and 8 for works")

['2', '4', '8', '8']

In [209]:
for m in p.finditer("24 hours a day; 8 in sleep and 8 for works"):
    print "\'%s\' found at %s" % (m.group(), m.span())

'2' found at (0, 1)
'4' found at (1, 2)
'8' found at (16, 17)
'8' found at (31, 32)


### implicit compile
methods like `search`, `match`, and `findall` can be called on module-level

In [210]:
m = re.findall("\d+", "1 round for Daan Park is 2350m")
print m

['1', '2350']


In [211]:
m = re.match("\d+", "1 round for Daan Park is 2350m")
print m.group()

1


In [212]:
m = re.search("\d+", "1 round for Daan Park is 2350m")
print m.group() # notice that only the first matched is returned

1


### more on `group` and `groups` method


In [213]:
m = re.match("([0-9]+)([a-z]+)", "123abc456")
# return the whole matched (sub)string
m.group()

'123abc'

In [214]:
# if grouper is used, they can be retrieved individually
m.group(0), m.group(1), m.group(2)

('123abc', '123', 'abc')

In [215]:
# or multiple retrieved
m.group(1,2,1)

('123', 'abc', '123')

In [216]:
# return each group matched in a tuple (works only if grouper is used)
m.groups() 

('123', 'abc')

In [217]:
# or in a dictionary
m.groupdict()

{}

(see the <a href="#/3">advanced section</a>)

### challenge!
>1 round for Daan Park is 2350m

=> try to retrieve two numbers present in the above sentence. (1 and 2350)

In [218]:
test_str = "1 round for Daan Park is 2350m"

In [219]:
m = re.match("(\d+).*(\d+).*", test_str)
m.groups() # failed to retrieve the second number (what's wrong?)

('1', '0')

In [220]:
m = re.match("(\d+)[^0-9]*(\d+).*", test_str)
m.groups() # repeater is GREEDY: they try to match as many as possible

('1', '2350')

### challenge!
> my code is 12
<br>
Mr. lucky7's code is 76
<br>
are you coded no.1?
<br>

=> retrieve all "code number" in this three lines

In [221]:
test_str = ["my code is 12", 
            "Mr. lucky7's code is 76",
            "are you coded no.1?"]

In [222]:
# hard-coded re
# useless, and sometimes dangerous!
p = re.compile("(12|76|1)")
[ p.search(s).group() for s in test_str ]

['12', '76', '1']

In [223]:
# match the number ended in the sentence, with optionally question mark
# still somewhat problem-specific and hence less general
# also notice that .findall only returns string matched in grouper, 
#     try "(\d+\??)" instead to see the diff
p = re.compile("(\d+)\??$")
[ p.findall(s)[-1] for s in test_str ]

['12', '76', '1']

In [224]:
# match all numbers and return the last one
p = re.compile("\d+")
[ p.findall(s)[-1] for s in test_str ]

['12', '76', '1']

### challenge!
> My name is Kyle Chung, loving buying at PChome.

=> retrieve first-char-capitalized words 

In [225]:
test_str = "My name is Kyle Chung, loving buying at PChome."

In [226]:
# bounder "\b" is essential here
# why double the escaper?
m = re.compile("\\b([A-Z][a-z]+)+")
m.findall(test_str)

['My', 'Kyle', 'Chung']

(see the section <a href="#/4">raw string notation</a> for more info about double escaping) 

### challenge!
> Things are getting HARDER! Now try capturing CamelUpperCased word(s).

=> retrieve any camel-cased word.

In [227]:
test_str = "Things are getting HARDER! Now try capturing CamelUpperCased word(s)."

In [228]:
m = re.compile("\\b[A-Z]+[a-z]+")
m.findall(test_str)

['Things', 'Now', 'Camel']


## Regex in Python<span style="color:blue"> | Advanced</span>
regex extensions: `(?<pattern>)`

this section covers four extensions:
+ non-capturer: `(?:<pattern>)`
+ ahead-looker: `(?=<pattern>)` and `(?!<pattern>)`
+ named-grouper: `(?P<gname>)`
+ backreferer: `(?p=<gname>)`

### non-capturer: `(?:...)`
to match without returning matched

In [229]:
test_str = "fact.1: 1 round for Daan Park is 2350m"
m = re.search("(?:^.*: )(\d+)", test_str)
print "string returned by .group: \n\t" + str(m.group())
print "string returned by .groups: \n\t" + str(m.groups())

m = re.search("(^.*:)", test_str)
print m.groups()


string returned by .group: 
	fact.1: 1
string returned by .groups: 
	('1',)
('fact.1:',)


### back to the challenge!

In [230]:
test_str = ["my code is 12", 
            "Mr. lucky7's code is 76",
            "are you coded no.1?"]
p = re.compile("(?:code)[^0-9]*(\d+)")
[ p.search(s).groups() for s in test_str ]

[('12',), ('76',), ('1',)]

### ahead-looker: `(?=...)` and `(?!...)`
only to match when the asserted string is (not) matched first

### named-grouper: `(?P<gname>)`
a python-specific extension


### backreferer: `(?P=gname)`
yet anotehr python-specific extension

### Raw String Notation

## Reference
+ [Facebook official Graph API reference](https://developers.facebook.com/docs/graph-api/reference/)
+ [Python official re tutorial](https://docs.python.org/2/howto/regex.html)
+ regex visualizer / tester:
    + [Debuggex](https://www.debuggex.com)
    + [regex101](https://regex101.com/#python)